<a href="https://colab.research.google.com/github/nyxNyschd/schuetzinnen/blob/abeer_julia/tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import math
from collections import Counter
import sklearn as sk

In [2]:
first = """The competition concerns the delivery of asphalt works on municipal roads, streets, foot and cycle paths, 
pavements, squares, parking places, some private roads and any selected new installations that are operated and
maintained by the individual municipalities.The annual value of the procurement is estimated to be: 15 000 000 
NOK-30 000 000 NOK excluding VAT per year. The value is an estimate and, thus, not binding. See the tender 
documentation for further information."""
second = """The establishment of a provisional cable installations at Mosseporten transformation station.
Construction of a power line route in Moss.
Building and ground work in Mosseporten transformer station.
"The contract includes all work that needs to be carried out with an excavator or similar machinery and associated necessary works, in connection with archaeological records, surveys and excavations."
"The contract includes all work that needs to be carried out with an excavator or similar machinery and associated necessary works, in connection with archaeological records, surveys and excavations."
"""

In [3]:
first = first.split(" ")
second = second.split(" ")

In [4]:
total= set(first).union(set(second))
print(total)

{'roads', 'a', 'and,', 'excavator', 'concerns', 'records,', 'be:', '\ndocumentation', 'excluding', 'The', 'transformation', 'is', 'paths,', 'Moss.\nBuilding', 'NOK', 'an', 'roads,', 'thus,', 'establishment', 'works', 'includes', 'private', 'and', 'or', 'excavations."\n"The', 'some', '15', 'are', 'connection', 'any', 'that', 'competition', 'delivery', 'per', 'transformer', 'squares,', 'selected', '\npavements,', 'asphalt', 'VAT', 'cycle', 'municipalities.The', 'See', 'information.', 'installations', 'binding.', 'work', 'estimate', 'similar', 'associated', 'operated', 'necessary', 'on', 'the', 'streets,', 'parking', 'procurement', 'provisional', 'station.\n"The', 'cable', 'all', 'carried', 'to', 'further', 'places,', 'value', 'foot', 'with', 'line', 'needs', 'of', 'year.', 'machinery', 'excavations."\n', 'station.\nConstruction', 'out', 'be', 'at', 'by', 'power', 'for', '\nNOK-30', 'in', 'surveys', 'route', 'contract', 'Mosseporten', 'estimated', 'new', 'annual', 'ground', 'not', 'tender

In [7]:
wordDictA = dict.fromkeys(total, 0) 
wordDictB = dict.fromkeys(total, 0)
for word in first:
    wordDictA[word]+=1
    
for word in second:
    wordDictB[word]+=1
#put them in a dataframe and then view the result:
pd.DataFrame([wordDictA, wordDictB])

,roads,a,"and,",excavator,concerns,"records,",be:,\ndocumentation,excluding,The,transformation,is,"paths,",Moss.\nBuilding,NOK,an,"roads,","thus,",establishment,works,includes,private,and,or,"excavations.""\n""The",some,15,are,connection,any,that,competition,delivery,per,transformer,"squares,",selected,"\npavements,",asphalt,VAT,...,cable,all,carried,to,further,"places,",value,foot,with,line,needs,of,year.,machinery,"excavations.""\n",station.\nConstruction,out,be,at,by,power,for,\nNOK-30,in,surveys,route,contract,Mosseporten,estimated,new,annual,ground,not,tender,archaeological,individual,municipal,"works,",000,and\nmaintained
0,1,0,1,0,1,0,1,1,1,2,0,2,1,0,1,1,1,1,0,1,0,1,2,0,0,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,...,0,0,0,1,1,1,2,1,0,0,0,2,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,1,1,0,1,1,0,1,1,0,4,1
1,0,2,0,2,0,2,0,0,0,1,1,0,0,1,0,2,0,0,1,0,2,0,5,2,1,0,0,0,2,0,2,0,0,0,1,0,0,0,0,0,...,1,2,2,2,0,0,0,0,4,1,2,2,0,2,1,1,2,2,1,0,1,0,0,4,2,1,2,2,0,0,0,1,0,0,2,0,0,2,0,0


In [8]:
#Now writing the TF function:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict
#running our sentences through the tf function:
tfFirst = computeTF(wordDictA, first)
tfSecond = computeTF(wordDictB, second)
#Converting to dataframe for visualization
pd.DataFrame([tfFirst, tfSecond])

,roads,a,"and,",excavator,concerns,"records,",be:,\ndocumentation,excluding,The,transformation,is,"paths,",Moss.\nBuilding,NOK,an,"roads,","thus,",establishment,works,includes,private,and,or,"excavations.""\n""The",some,15,are,connection,any,that,competition,delivery,per,transformer,"squares,",selected,"\npavements,",asphalt,VAT,...,cable,all,carried,to,further,"places,",value,foot,with,line,needs,of,year.,machinery,"excavations.""\n",station.\nConstruction,out,be,at,by,power,for,\nNOK-30,in,surveys,route,contract,Mosseporten,estimated,new,annual,ground,not,tender,archaeological,individual,municipal,"works,",000,and\nmaintained
0,0.013889,0.000000,0.013889,0.000000,0.013889,0.000000,0.013889,0.013889,0.013889,0.027778,0.000000,0.027778,0.013889,0.000000,0.013889,0.013889,0.013889,0.013889,0.000000,0.013889,0.000000,0.013889,0.027778,0.000000,0.000000,0.013889,0.013889,0.013889,0.000000,0.013889,0.013889,0.013889,0.013889,0.013889,0.000000,0.013889,0.013889,0.013889,0.013889,0.013889,...,0.000000,0.000000,0.000000,0.013889,0.013889,0.013889,0.027778,0.013889,0.000000,0.000000,0.000000,0.027778,0.013889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,0.000000,0.013889,0.013889,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,0.013889,0.013889,0.000000,0.013889,0.013889,0.000000,0.013889,0.013889,0.000000,0.055556,0.013889
1,0.000000,0.024691,0.000000,0.024691,0.000000,0.024691,0.000000,0.000000,0.000000,0.012346,0.012346,0.000000,0.000000,0.012346,0.000000,0.024691,0.000000,0.000000,0.012346,0.000000,0.024691,0.000000,0.061728,0.024691,0.012346,0.000000,0.000000,0.000000,0.024691,0.000000,0.024691,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012346,0.024691,0.024691,0.024691,0.000000,0.000000,0.000000,0.000000,0.049383,0.012346,0.024691,0.024691,0.000000,0.024691,0.012346,0.012346,0.024691,0.024691,0.012346,0.000000,0.012346,0.000000,0.000000,0.049383,0.024691,0.012346,0.024691,0.024691,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.024691,0.000000,0.000000,0.024691,0.000000,0.000000


In [11]:
#creating the log portion of the Excel table we saw earlier
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict
#inputing our sentences in the log file
idfs = computeIDF([wordDictA, wordDictB])
#The actual calculation of TF*IDF from the table above:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf
#running our two sentences through the IDF:
idfFirst = computeTFIDF(tfFirst, idfs)
idfSecond = computeTFIDF(tfSecond, idfs)
#putting it in a dataframe
pd.DataFrame([idfFirst, idfSecond])

,roads,a,"and,",excavator,concerns,"records,",be:,\ndocumentation,excluding,The,transformation,is,"paths,",Moss.\nBuilding,NOK,an,"roads,","thus,",establishment,works,includes,private,and,or,"excavations.""\n""The",some,15,are,connection,any,that,competition,delivery,per,transformer,"squares,",selected,"\npavements,",asphalt,VAT,...,cable,all,carried,to,further,"places,",value,foot,with,line,needs,of,year.,machinery,"excavations.""\n",station.\nConstruction,out,be,at,by,power,for,\nNOK-30,in,surveys,route,contract,Mosseporten,estimated,new,annual,ground,not,tender,archaeological,individual,municipal,"works,",000,and\nmaintained
0,0.004181,0.000000,0.004181,0.000000,0.004181,0.000000,0.004181,0.004181,0.004181,0.0,0.000000,0.008362,0.004181,0.000000,0.004181,0.0,0.004181,0.004181,0.000000,0.004181,0.000000,0.004181,0.0,0.000000,0.000000,0.004181,0.004181,0.004181,0.000000,0.004181,0.0,0.004181,0.004181,0.004181,0.000000,0.004181,0.004181,0.004181,0.004181,0.004181,...,0.000000,0.000000,0.000000,0.0,0.004181,0.004181,0.008362,0.004181,0.000000,0.000000,0.000000,0.0,0.004181,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004181,0.000000,0.004181,0.004181,0.000000,0.000000,0.000000,0.000000,0.000000,0.004181,0.004181,0.004181,0.000000,0.004181,0.004181,0.000000,0.004181,0.004181,0.000000,0.016724,0.004181
1,0.000000,0.007433,0.000000,0.007433,0.000000,0.007433,0.000000,0.000000,0.000000,0.0,0.003716,0.000000,0.000000,0.003716,0.000000,0.0,0.000000,0.000000,0.003716,0.000000,0.007433,0.000000,0.0,0.007433,0.003716,0.000000,0.000000,0.000000,0.007433,0.000000,0.0,0.000000,0.000000,0.000000,0.003716,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003716,0.007433,0.007433,0.0,0.000000,0.000000,0.000000,0.000000,0.014866,0.003716,0.007433,0.0,0.000000,0.007433,0.003716,0.003716,0.007433,0.007433,0.003716,0.000000,0.003716,0.000000,0.000000,0.014866,0.007433,0.003716,0.007433,0.007433,0.000000,0.000000,0.000000,0.003716,0.000000,0.000000,0.007433,0.000000,0.000000,0.007433,0.000000,0.000000


Alternative Methode mit sklearn

In [12]:
#first step is to import the library
from sklearn.feature_extraction.text import TfidfVectorizer
#for the sentence, make sure all words are lowercase or you will run into error. for simplicity, I just made the same sentence all lowercase
firstV= 'the car is driven on the road'
secondV= 'the truck is driven on the highway'
#calling the TfidfVectorizer
vectorize= TfidfVectorizer()
#fitting the model and passing our sentences right away:
response= vectorize.fit_transform([firstV, secondV])
print(response)

  (0, 5)	0.42471718586982765
  (0, 4)	0.30218977576862155
  (0, 1)	0.30218977576862155
  (0, 3)	0.30218977576862155
  (0, 0)	0.42471718586982765
  (0, 6)	0.6043795515372431
  (1, 2)	0.42471718586982765
  (1, 7)	0.42471718586982765
  (1, 4)	0.30218977576862155
  (1, 1)	0.30218977576862155
  (1, 3)	0.30218977576862155
  (1, 6)	0.6043795515372431
